## Naïve Bayes – Additional Advice
This notebook guides you through multiple steps you can follow to create a naïve Bayes classifier. After following these steps you will still need to collate and move your code into the main assignment notebook file so that it meets the required format.

Read each step (including the maths!) carefully.

You can implement a naïve Bayes classifier without following this advice.

This notebook will not be graded and does not need to be submitted.

In [24]:
import numpy as np

training_spam = np.loadtxt(open("training_spam.csv"), delimiter=",")
print("Shape of the spam training data set:", training_spam.shape)
print(training_spam)

#training_spam = np.loadtxt(open("testing_spam.csv"), delimiter=",").astype(int)
#print("Shape of the spam testing data set:", training_spam.shape)
#print(training_spam)

Shape of the spam training data set: (1000, 55)
[[1. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 1. 0. 0.]
 [0. 0. 0. ... 1. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [1. 1. 1. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 1. 1.]]


## The model:  naïve Bayes
Your [naïve Bayes](https://en.wikipedia.org/wiki/Naive_Bayes_classifier) classifier will distinguish between two classes:

* $C = 1$ for spam messages
* $C = 0$ for ham messages


The classifier builds a model for the probability $p(C=c\ |\ \text{message})$ that a given message belongs to a certain class. A new message is then classified based on the Bayesian *maximum a posteriori* estimate
$\require{color}$
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \  \textcolor{blue}{p(C=c\ |\ \text{message})}.
\end{equation}
Using Bayes' rule we can write

\begin{equation}
p(C=c\ |\ \text{message}) = \frac{p(\text{message}\ |\ C=c)p(C=c)}{p(\text{message}\ |\ C=1)p(C=1) + p(\text{message}\ |\ C=0)p(C=0)}.  \quad \quad 
\end{equation}

The denominator is the same for both classes and we can thus drop it to get

\begin{equation}
\textcolor{blue}{p(C=c\ |\ \text{message})} \propto \textcolor{orange}{p(\text{message}\ |\ C=c)}\textcolor{green}{p(C=c)},
\end{equation}

where $\propto$ means "proportional to". The class priors $\textcolor{green}{p(C=c)}$ can be computed directly (you will do so in exercise A) but we need to further simplify $\textcolor{orange}{p(\text{message} \ |\ C=c)}$.


### Choice of the event model: *Multinomial* naïve Bayes

Different naïve Bayes models differ in their distributional assumptions about $\textcolor{orange}{p(\text{message}\ |\ C=c)}$. We represent a message using a **binary** [bag-of-words](https://en.wikipedia.org/wiki/Bag-of-words_model) model. Specifically, a message is represented as a set of $k$ keywords, that is, $message = (w_1, ..., w_k)$, where $w_i = 1$ if the  keyword $w_i$ appears in the message and $w_i = 0$ otherwise.

We assume that the $p(w_1, ..., w_k |\ C=c)$ follows a [multinomial distribution](https://en.wikipedia.org/wiki/Multinomial_distribution) for each class. Don't let the name scare you, this model simply assigns probabilities to different counts of events with multiple outcomes. So for example: "I roll a biased six-sided die six times, what is the probability that I get each side occurring exactly once" is a question that can be answered with a multinomial distribution. You don't need to understand all of the equations on the Wikipedia page.

Intuitively, the multinomial distribution assumes that the words of a message were "drawn" independently from a bag of $k$ different words. Depending on the class membership $c$, each keyword $w$ has a probability $\theta_{c, w}$ of being drawn. For example,

* $\theta_{spam, w}$ will have high value for $w \in \{$bank, transfer, buy,... $\}$.
* $\theta_{ham, w}$ will have high value for $w \in \{$paper, conference, proposal, experiment,... $\}$, if the training data was mostly gathered from emails of researchers.

Under these assumptions, the likelihood of a message, given that it belongs to class $c$, is then proportional to
\begin{equation}
\textcolor{orange}{p(\text{message}\ |\ C=c)} \propto \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}.
\end{equation}


The parameters $\textcolor{brown}{\theta_{c, w}}$ are estimated by counting the relative frequencies in the training data. Use **Laplace-smoothing** with $\alpha = 1$ (add-one smoothing), that is,
\begin{equation}
\textcolor{brown}{\theta_{c, w}} = \frac{n_{c, w} + \alpha}{n_{c} + k \alpha},
\end{equation}
where $n_{c, w}$ is the number of times the keyword $w$ appears in messages of class $c$ in the training set and $n_{c}$ is the total count of keywords for all messages of class $c$, that is, $n_{c} = \sum_w n_{c, w}$.



We are now finally able to rewrite the *maximum a posteriori* estimate in a form that is easy to compute:
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}} \right)^{w_i}\right].
\end{equation}


#### Increasing numerical stability
We can increase the numerical stability of the algorithm by taking logarithms of the posterior distributions, that is,
\begin{equation}
\hat{c} = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \log \left( \textcolor{green}{p(C=c)}   \prod_{i = 1}^k  \left(\textcolor{brown}{\theta_{c, w_i}}\right)^{w_i} \right) \\
 = \underset{c \in \{0,1\}}{\operatorname{argmax}} \ \left[ \log( \textcolor{green}{p(C=c)}) + \sum_{i = 1}^k w_i \ \log \left(\textcolor{brown}{\theta_{c, w_i}} \right) \right].
\end{equation}

## Part A: Estimate class priors

Define a function called `estimate_log_class_priors()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns a numpy array containing the **the logarithm** of the empirical class priors $\textcolor{green}{p(C=c)}$ for $c \in \{0, 1\}$.

In [33]:
def estimate_log_class_priors(data):#(self, data):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column, calculate the logarithm of the empirical class priors,
    that is, the logarithm of the proportions of 0s and 1s:
        log(p(C=0)) and log(p(C=1))

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]
                 the first column contains the binary response (coded as 0s and 1s).

    :return log_class_priors: a numpy array of length two
    """

    num_rows = data.shape[0]
    count_c1 = np.sum(data[:, 0])
    
    # print("P0: {}, P1: {}", ((num_rows - count_c1) / num_rows),(count_c1 / num_rows))
    log_c0_prior = math.log((num_rows - count_c1) / num_rows)
    log_c1_prior = math.log(count_c1 / num_rows)
    log_class_priors = np.array([log_c0_prior, log_c1_prior])
    
    #self.log_class_priors = log_class_priors
    return log_class_priors

In [34]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_priors = estimate_log_class_priors(training_spam)
print("result", log_class_priors)

# Check length
assert(len(log_class_priors) == 2)

# Check whether the returned object is a numpy.ndarray
assert(isinstance(log_class_priors, np.ndarray))

# Check wehther the values of this numpy.array are floats.
assert(log_class_priors.dtype == float)

# Check wehther the values are both negative (the logarithm of a probability 0 < p < 1 should be negative).
assert(np.all(log_class_priors < 0))

result [-0.48939034 -0.94933059]


## Part B: Estimate class-conditional likelihoods
Define a function called `estimate_log_class_conditional_likelihoods()` that takes as input a data set with binary response variable (0s and 1s) in the left-most column and returns **the logarithm** of the empirical class-conditional likelihoods $\log \left(\textcolor{brown}{\theta_{c, w_i}} \right)$ for all words $w_i$ and both classes ($c \in {0, 1}$). These parameters should be returned in a two-dimensional numpy-array with shape = `[num_classes, num_features]`.

Assume a multinomial event model and use Laplace smoothing with $\alpha = 1$. 

Hint: many `numpy`-functions contain an `axis` argument. If you specify `axis=0`, you can perform column-wise (that is, feature-wise!) computations.

In [12]:
def estimate_log_class_conditional_likelihoods(data, alpha=1.0):
    """
    Given a data set with binary response variable (0s and 1s) in the
    left-most column and binary features (words), calculate the empirical
    class-conditional likelihoods, that is,
    log(P(w_i | c)) for all features w_i and both classes (c in {0, 1}).

    Assume a multinomial feature distribution and use Laplace smoothing
    if alpha > 0.

    :param data: a two-dimensional numpy-array with shape = [n_samples, 1 + n_features]

    :return theta:
        a numpy array of shape = [2, n_features]. theta[j, i] corresponds to the
        logarithm of the probability of feature i appearing in a sample belonging 
        to class j.
    """
    import math
    
   
    ##############################################
    #put together to create a 1D array of the sum of each element where C=1 (tested and correct in excel)
    c1_rows = data[np.nonzero(data[:,0]),1:]
    c1_numrows = c1_rows[0,:,0].size
    #print("c1_numrows: ", c1_numrows)
    c1_element_sums = np.array(np.sum(c1_rows[0,:], axis = 0))
    if alpha > 0:
        c1_element_sums = np.add(1, c1_element_sums) # add 1 to every element fpr laplace smoothing
    #print("c1_element_sums,",c1_element_sums)
    c1_element_eccl = np.log(np.divide(c1_element_sums , c1_numrows))
    #print("c1_element_eccl: ", c1_element_eccl, c1_element_eccl.shape)
    #print("Test numpy log10 0.147: ", np.log10(0.147))
    #print("Test math log 0.147: ", math.log(0.147))
    ##################################################
    
    
    #THINK I HAVE THIS WRONG - I SHOULD BE COUNTING THE PROBABILITY OF EACH ELEMENT BEING 1 FOR A CLASS, BUT THINK I AM COUNTING THE ELEMENTS WHICH ARE ZERO FOR EACH CLASS
    #now get rows where C=0 using idea from https://stackoverflow.com/questions/4588628/find-indices-of-elements-equal-to-zero-in-a-numpy-array
    #################################################
    c0_rows = data[np.where(data[:,0]==0)[0],1:]
    #with np.printoptions(threshold=np.inf):
    #    print("c0_rows", c0_rows.shape, "\n", c0_rows)
    c0_numrows = c0_rows[:,0].size
    #print("c0_numrows: ", c0_numrows)
    c0_element_sums = np.array(np.sum(c0_rows, axis = 0))
    if alpha > 0:
        c0_element_sums = np.add(1, c0_element_sums) # add 1 to every element fpr laplace smoothing
    #print("c0_element_sums,",c0_element_sums)
    c0_element_eccl = np.log(np.divide(c0_element_sums , c0_numrows))
    #print("c0_element_eccl: ", c0_element_eccl, c0_element_eccl.shape)
    
    theta = np.array([c0_element_eccl,c1_element_eccl])
    #print("theta: ", theta, theta.shape)
    
    return theta


In [13]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
log_class_conditional_likelihoods = estimate_log_class_conditional_likelihoods(training_spam, alpha=1.0)
print(log_class_conditional_likelihoods)

# Check data type(s)
assert(isinstance(log_class_conditional_likelihoods, np.ndarray))

# Check shape of numpy array
assert(log_class_conditional_likelihoods.shape == (2, 54))

# Check data type of array elements
assert(log_class_conditional_likelihoods.dtype == float)

[[-1.7455361  -2.30749107 -1.2825665  -5.72521776 -1.47672251 -2.25948185
  -3.85341558 -2.70479287 -2.42938089 -1.79339212 -2.78077878 -0.84241583
  -2.02391578 -3.08616043 -3.93345829 -2.30749107 -2.24397767 -2.02391578
  -0.57772328 -3.85341558 -1.07125741 -4.62660547 -3.58515159 -3.85341558
  -0.97162756 -1.27087046 -1.31849851 -1.85401674 -2.02391578 -1.90750543
  -2.29123055 -2.54716393 -2.19885723 -2.52654464 -1.86448804 -1.78363595
  -1.24788094 -3.93345829 -2.16986969 -2.30749107 -3.08616043 -2.0876316
  -2.24397767 -2.29123055 -1.2198679  -1.76440459 -3.93345829 -2.80744702
  -1.69097712 -0.58941932 -1.91855527 -1.23658139 -2.25948185 -2.44807302]
 [-0.96799211 -1.16263415 -0.51168732 -4.01251454 -0.42109043 -0.87702033
  -0.89582966 -1.01678227 -1.2048345  -0.74348894 -1.2048345  -0.46536325
  -1.13011096 -2.06660439 -1.95109151 -0.56026199 -0.82262626 -1.04576981
  -0.10622221 -1.48108788 -0.20585205 -3.01398571 -1.11423761 -0.90856869
  -3.47351804 -3.76120012 -5.26527751 

## Part  C: Classify e-mails

Having calculated the log class priors and the log class-conditional likelihoods for a given training set, define a function called `predict()`that takes a data set of new messages as input and predicts for each message whether it is spam or not. Note that the input should **not** contain a response variable.

In [14]:
import math

def get_probability(class_num, message, log_class_priors, log_class_conditional_likelihoods):
    print("log_class_conditional_likelihoods shape: ", log_class_conditional_likelihoods.shape)
    print("message shape: ", message.shape)
    running_probability = 1
    i=0
    for element in message:
        if element == 1:
            running_probability *= log_class_conditional_likelihoods[class_num,i]
        else:
            #print("original probability: ", log_class_conditional_likelihoods[class_num,i])
            #print("inverse log:", math.e ** log_class_conditional_likelihoods[class_num,i])
            #print("opposite probability:", 1-(math.e ** log_class_conditional_likelihoods[class_num,i]))
            #print("opposite log probability:", math.log(1-(math.e ** log_class_conditional_likelihoods[class_num,i])))
            running_probability *= math.log(1-(math.e ** log_class_conditional_likelihoods[class_num,i]))
        i+=1
    
    denominator = log_class_priors[class_num]
    #print("denominator:", denominator)
    #perform calculation
    probability = (running_probability / denominator)
    return probability

def predict(new_data, log_class_priors, log_class_conditional_likelihoods):
    """
    Given a new data set with binary features, predict the corresponding
    response for each instance (row) of the new_data set.

    :param new_data: a two-dimensional numpy-array with shape = [n_test_samples, n_features].
    :param log_class_priors: a numpy array of length 2.
    :param log_class_conditional_likelihoods: a numpy array of shape = [2, n_features].
        theta[j, i] corresponds to the logarithm of the probability of feature i appearing
        in a sample belonging to class j.
    :return class_predictions: a numpy array containing the class predictions for each row
        of new_data.
    """
    ### YOUR CODE HERE...
    class_predictions = np.empty(new_data.shape[0])
    
    message = 0
    for row in new_data:
        print("Row: ", row)
        #calculate numerator
            # note from self consider making more accurate by using the inverse of the probability if a selection is NOT true for a feature and class
        class0_probability = get_probability(0, row, log_class_priors, log_class_conditional_likelihoods)
        print("class0_probability: ", class0_probability)
        class1_probability = get_probability(1, row, log_class_priors, log_class_conditional_likelihoods)
        print("class1_probability: ", class1_probability)
        probably_spam = class1_probability > class0_probability
        print("probably_spam: ", probably_spam)
        
#        running_probability_1 = 1
#        i=0
#        for element in row:
#            if element == 1:
#                running_probability_1 *= log_class_conditional_likelihoods[1,i]
#            i+=1

    #get log class prior (denominator)
#        denominator = log_class_priors[1]
#        print("denominator:", denominator)
        #perform calculation
#        probability = (running_probability / denominator)
        #append to class predicions
 #       print("probability: ", probability)
    
    #class_predictions = np.append(class_predictions, probability)
    
        class_predictions[message] = probably_spam
        message += 1
        
        #COMMENT OUT LATER
        #if message > 1:
        #    return class_predictions
    
    return class_predictions


In [15]:
# You can use this cell to check whether the returned objects of your function are of the right data type.
class_predictions = predict(training_spam[:, 1:], log_class_priors, log_class_conditional_likelihoods)

print("class_predictions", class_predictions)

# Check data type(s)
assert(isinstance(class_predictions, np.ndarray))

# Check shape of numpy array
print("class_predictions.shape:", class_predictions.shape)
assert(class_predictions.shape == (1000,))

# Check data type of array elements
assert(np.all(np.logical_or(class_predictions == 0, class_predictions == 1)))

Row:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 0. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.5470389064933357e-51
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.6950698196967364e-54
probably_spam:  True
Row:  [0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -7.500846071871016e-48
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -5.510567472945593e-56
probably_spam:  True
Row:  [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.

Row:  [1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 1. 1. 1.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.318555701087363e-25
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -4.7156058016628184e-48
probably_spam:  True
Row:  [0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -8.804320081969813e-38
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.2679958017205207e-52
probably_spam:  True
Row:  [0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0.
 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0.

class0_probability:  -2.924291135403224e-47
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -3.0520186924046206e-49
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0.
 0. 1. 0. 1. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -5.627503209608588e-46
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -5.82455894408855e-47
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -4.9234795357313206e-49
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.965350694

log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -4.828020197227269e-46
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -4.59660294328298e-48
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 0.
 0. 0. 1. 0. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -5.883432203333984e-45
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -6.44783100621353e-42
probably_spam:  False
Row:  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 0. 1. 1. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -1.0273403059713534e-43
log_class_conditional_likelihoods 

class1_probability:  -1.4409822746011572e-54
probably_spam:  True
Row:  [1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0.
 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 1. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.4589738438368766e-43
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -9.56382478516116e-53
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.4319900108432854e-52
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -5.085797229607526e-53
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 

 1. 1. 1. 1. 1. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -9.601093034256546e-18
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.1953703519576562e-46
probably_spam:  True
Row:  [1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0.
 1. 1. 1. 1. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -9.963589699523398e-41
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.90100514016645e-46
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0.
 0. 0. 0. 0. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.8851495680454766e-47
log_class_con

class1_probability:  -1.6472569135232806e-53
probably_spam:  True
Row:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -2.4319900108432854e-52
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -5.085797229607526e-53
probably_spam:  True
Row:  [0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
 0. 1. 0. 1. 0. 0.]
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class0_probability:  -6.668175694804712e-43
log_class_conditional_likelihoods shape:  (2, 54)
message shape:  (54,)
class1_probability:  -1.5309507896060083e-53
probably_spam:  True
Row:  [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
 0.

Now test your `predict` function by classifying messages. You can do this to the *training* data, but you should also try it on the *testing* data. 

In [16]:
# Check accuracy
true_classes = training_spam[:, 0]
training_set_accuracy = np.mean(np.equal(class_predictions, true_classes))
print(f"Accuracy on the training set: {training_set_accuracy}")

Accuracy on the training set: 0.632


Once you are done, you can move the code into the main assignment notebook.

One way to do this is to follow the rough structure of the class that already exists in that notebook. You can use the `train` method to pass in the data and perform all of the steps before the prediction. You should store data in instance variables, e.g. `self.log_class_priors` and `self.log_class_conditional_likelihoods`. This means that then you can set up the `predict` method to match the one above without needing to pass in the additional variables. **Important:** the predict method must only take a single variable as a parameter (the one called `new_data`) in the skeleton code above.